**Credit to Seneth**

In [4]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.metrics as skmetrics
from sklearn.metrics import mean_squared_error

import warnings                               
warnings.filterwarnings('ignore')

In [2]:
profit = pd.read_csv('profit.csv')['Profit']
train, test = profit[:-3], profit[-3:]

**Method 1: direct multi-step forecast**

In [5]:
def evaluate_models_direct_cv_step(dataset, p_values, d_values, q_values, step, k_fold):
    result = []
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                validation_size=step
                train_size=len(dataset)-k_fold*step
                step1=[]
                step2=[]
                step3=[]
                stepavg = []
                for k in range(0,k_fold):
                    train, test = dataset[0:train_size+k*validation_size], \
                            dataset[train_size+k*validation_size:train_size+k*validation_size+step]
                    model = ARIMA(train, order=order) 
                    model_fit = model.fit()
                    prediction=model_fit.forecast(step)

                    # Step 1
                    rmse1 = skmetrics.mean_squared_error(test[:1], prediction[:1])
                    step1.append(rmse1)

                    # Step 2
                    rmse2 = skmetrics.mean_squared_error(test[1:2], prediction[1:2])
                    step2.append(rmse2)
                    
                    # Step 3
                    rmse3 = skmetrics.mean_squared_error(test[2:3], prediction[2:3])
                    step3.append(rmse3)

                    rmseavg = np.sqrt(skmetrics.mean_squared_error(test, prediction))
                    stepavg.append(rmseavg)
                    
                rmse1_avg=np.sqrt(sum(step1)/k_fold)
                rmse2_avg=np.sqrt(sum(step2)/k_fold)
                rmse3_avg=np.sqrt(sum(step3)/k_fold)
                rmse_avg = sum(stepavg)/k_fold
                result.append((order, rmse1_avg, rmse2_avg, rmse3_avg, rmse_avg))
                
    min_step1 = min(result, key=lambda x: x[1])
    print(f"Best Step1 & RMSE: {min_step1[0], min_step1[1]}")
    min_step2 = min(result, key=lambda x: x[2])
    print(f"Best Step2 & RMSE: {min_step2[0], min_step2[2]}")
    min_step3 = min(result, key=lambda x: x[3])
    print(f"Best Step3 & RMSE: {min_step3[0], min_step3[3]}")
    min_avg = min(result, key=lambda x: x[4])
    print(f"Best AVG & RMSE: {min_avg[0], min_avg[4]}")
    
p_values = range(0, 6)
d_values = range(0, 1) 
q_values = range(0, 6)
evaluate_models_direct_cv_step(train, p_values, d_values, q_values, 3, 5)

Best Step1 & RMSE: ((0, 0, 0), 0.8536881722355835)
Best Step2 & RMSE: ((0, 0, 0), 0.6987338580443837)
Best Step3 & RMSE: ((0, 0, 4), 0.7352487743097409)
Best AVG & RMSE: ((0, 0, 0), 0.7931228819385543)


**Method 2: recursive multi-step forecast**

In [6]:
def evaluate_models_recursive_cv(dataset, p_values, d_values, q_values, step, k_fold):
    result = []
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                validation_size=step
                train_size=len(dataset)-k_fold*step
                rmse1=0
                rmse2=0
                rmse3=0
                rmseavg = 0
                for k in range(0,k_fold):
                    train, test = dataset[0:train_size+k*validation_size], \
                        dataset[train_size+k*validation_size:train_size+k*validation_size+3]
                    model = ARIMA(train, order=order) 
                    model_fit = model.fit()
                    prediction1 = model_fit.forecast(1)
                    rmse1 = rmse1 + skmetrics.mean_squared_error(test[:1], prediction1)

                    train = train.append(prediction1)  # Add prior prediction to training set 
                    model = ARIMA(train, order=order)
                    model_fit = model.fit()
                    prediction2 = model_fit.forecast(1)
                    rmse2 = rmse2 + skmetrics.mean_squared_error(test[1:2], prediction2)

                    train = train.append(prediction2)  # Add prior prediction to training set 
                    model = ARIMA(train, order=order)
                    model_fit = model.fit()
                    prediction3 = model_fit.forecast(1)
                    rmse3 = rmse3 + skmetrics.mean_squared_error(test[2:], prediction3)


                    rmseavg = rmseavg + np.sqrt((skmetrics.mean_squared_error(test[:1], prediction1) \
                                                 + skmetrics.mean_squared_error(test[1:2], prediction2) \
                                                 + skmetrics.mean_squared_error(test[2:], prediction3))/3)
                    
                rmse1_avg=np.sqrt(rmse1/k_fold)
                rmse2_avg=np.sqrt(rmse2/k_fold)
                rmse3_avg=np.sqrt(rmse3/k_fold)
                rmse_avg = rmseavg/k_fold
                result.append((order, rmse1_avg, rmse2_avg, rmse3_avg, rmse_avg))
                
    min_step1 = min(result, key=lambda x: x[1])
    print(f"Best Step1 & RMSE: {min_step1[0], min_step1[1]}")
    min_step2 = min(result, key=lambda x: x[2])
    print(f"Best Step2 & RMSE: {min_step2[0], min_step2[2]}")
    min_step3 = min(result, key=lambda x: x[3])
    print(f"Best Step3 & RMSE: {min_step3[0], min_step3[3]}")
    min_avg = min(result, key=lambda x: x[4])
    print(f"Best AVG & RMSE: {min_avg[0], min_avg[4]}")


In [7]:
p_values = range(0, 6)
d_values = range(0, 1) 
q_values = range(0, 6)
evaluate_models_recursive_cv(train, p_values, d_values, q_values, 3, 5)


Best Step1 & RMSE: ((0, 0, 0), 0.8536881722355835)
Best Step2 & RMSE: ((0, 0, 0), 0.6987338693536749)
Best Step3 & RMSE: ((0, 0, 4), 0.7352481070053898)
Best AVG & RMSE: ((0, 0, 0), 0.7931228710218959)
